In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

import numpy as np
import json

In [37]:
batch_size  = 32
epochs      = 50
latent_dim  = 128

In [40]:
# Get training/testing data 

filepath = 'D://NLP/translation2019zh/translation2019zh_train.json'


input_texts  = []
target_texts = []

input_characters  = set()
target_characters = set()


fd = open(filepath, 'r', encoding='utf-8')

i = 0
lines = fd.readlines()

for line in lines:
    i += 1
    if i == 512:
        break
    
    line = line.strip('{').strip('}\n')
    
    input_text   = line.split('"english": ')[1].split(', "chinese": ')[0].strip('"')
    target_text  = line.split(', "chinese": ')[1].strip('"')
    target_text  = '\t' + target_text  + '\n'
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
            
# 字典排序            
input_characters  = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

# 計算編碼器、解碼器的最大長度
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

# 以dict儲存字典單字及序號
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# 設定編碼器、解碼器input起始值(均為0矩陣)
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


# 設定 encoder_input、decoder_input對應的順序    
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.

    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

Number of samples: 511
Number of unique input tokens: 126
Number of unique output tokens: 2002
Max sequence length for inputs: 232
Max sequence length for outputs: 122
